# **TV Programs**

For a list of methods for this resource, see the end of this page.
## Resource representations

The metadata for a program.

```python
{
  "id": string,
  "original_title": string,
  "local_title": string,
  "year": string,
  "start_date": string,
  "start_time": string,
  "duration_in_seconds": string,
  "channel": {
    "id": string,
    "name": string,
    "country": string
  }
}
```

In [1]:
! python -V
! python -m django --version
! ls .

Python 2.7.17 :: Anaconda, Inc.
1.11.28
Programs API.ipynb      upload_from_csv.py      vericast_reports.py
__init__.py             upload_from_csv.pyc
__init__.pyc            uploading_epg_csv.ipynb


In [2]:
import os
os.chdir('../')

In [3]:
! ./manage.py makemigrations
# ! ./manage.py migrate tv zero
! ./manage.py migrate --run-syncdb

No changes detected
Operations to perform:
  Synchronize unmigrated apps: core, crispy_forms, debug_toolbar, django_extensions, messages, rest_framework, staticfiles
  Apply all migrations: admin, auth, contenttypes, sessions, tv
Synchronizing apps without migrations:
  Creating tables...
    Running deferred SQL...
Running migrations:
  No migrations to apply.


In [4]:
import io
from tv.models import Channel, Program, Showtime
from tv.serializers import ChannelSerializer, ProgramSerializer, ShowtimeSerializer
from rest_framework.renderers import JSONRenderer
from rest_framework.parsers import JSONParser

In [5]:
channel = Channel.objects.get(pk=1)
channel.get_country_code_display()

u'France'

In [6]:
channel = Channel.objects.get(pk=1)
print(type(channel.name))
print(type(channel.id))
print(type(channel.country_code))
# For hyperlinked serializers
# serializer = ChannelSerializer(channel, context={'request': request})
serializer = ChannelSerializer(channel)
print(serializer.data)
content = JSONRenderer().render(serializer.data)
print(content)
stream = io.BytesIO(content)
data = JSONParser().parse(stream)
print(data)

<type 'unicode'>
<type 'int'>
<type 'unicode'>
{'country': u'France', 'uid': 384303590, 'name': u'HD1 (France)'}
{"uid":384303590,"name":"HD1 (France)","country":"France"}
{u'country': u'France', u'uid': 384303590, u'name': u'HD1 (France)'}


In [7]:
serializer = ChannelSerializer(data=data)
print(type(data["country"]))
print(type(data["uid"]))
print(type(data["name"]))
serializer.is_valid()

<type 'unicode'>
<type 'int'>
<type 'unicode'>


True

## Program serializers

In [8]:
channel = ChannelSerializer(channel)
program = Program.objects.get(pk=441)
print(type(program.local_title))
print(type(program.uid))
print(type(program.channel))
# For hyperlinked serializers
# serializer = ChannelSerializer(channel, context={'request': request})
serializer = ProgramSerializer(program)
serializer.data


<type 'unicode'>
<type 'int'>
<class 'tv.models.Channel'>


{u'year': u'2017', u'original_title': u'David Bowie: The Last Five Years', u'uid': 4413966562, u'channel': OrderedDict([('uid', 69036687), ('name', u'RTE 1'), ('country', u'Ireland')]), u'local_title': u'David Bowie: The Last Five Years'}

In [9]:
content = JSONRenderer().render(serializer.data)
content

'{"uid":4413966562,"original_title":"David Bowie: The Last Five Years","local_title":"David Bowie: The Last Five Years","year":"2017","channel":{"uid":69036687,"name":"RTE 1","country":"Ireland"}}'

In [10]:
import io

stream = io.BytesIO(content)
data = JSONParser().parse(stream)
data['uid']=99999

In [11]:
serializer = ProgramSerializer(data=data)
serializer.is_valid()

True

In [12]:
program = Program.objects.get(pk=441)
showtime = Showtime.objects.get(pk=1)
print(type(showtime.program))
print(type(showtime.start_time))
# For hyperlinked serializers
# serializer = ChannelSerializer(channel, context={'request': request})
serializer = ShowtimeSerializer(showtime)
print(serializer.data)
content = JSONRenderer().render(serializer.data)
print(content)
stream = io.BytesIO(content)
data = JSONParser().parse(stream)
data['program']['uid']=99999
print(data)

<class 'tv.models.Program'>
<type 'datetime.datetime'>
{u'duration': 84000.0, u'start_time': '00:00', u'program': OrderedDict([(u'uid', 461044), (u'original_title', u'\xc0 communiquer'), (u'local_title', u'\xc0 communiquer'), (u'year', u'n/a'), (u'channel', OrderedDict([('uid', 384303590), ('name', u'HD1 (France)'), ('country', u'France')]))]), u'start_date': '2018-01-01'}
{"program":{"uid":461044,"original_title":"À communiquer","local_title":"À communiquer","year":"n/a","channel":{"uid":384303590,"name":"HD1 (France)","country":"France"}},"start_date":"2018-01-01","start_time":"00:00","duration":84000.0}
{u'duration': 84000.0, u'start_time': u'00:00', u'program': {u'year': u'n/a', u'original_title': u'\xc0 communiquer', u'uid': 99999, u'channel': {u'country': u'France', u'uid': 384303590, u'name': u'HD1 (France)'}, u'local_title': u'\xc0 communiquer'}, u'start_date': u'2018-01-01'}


In [13]:
serializer = ShowtimeSerializer(data=data)
serializer.is_valid()
serializer.errors

{}

In [18]:
! curl -H 'Accept: application/json; indent=4' -u operations:1comando http://www.bmatapis.com:8000/tv/v1/programs/1134/?format=json

{
    "program": {
        "uid": 11742116,
        "original_title": "Minutos musicales",
        "local_title": "Minutos musicales",
        "year": "n/a",
        "channel": {
            "uid": 69036190,
            "name": "La Sexta",
            "country": "Spain"
        }
    },
    "start_date": "2018-01-15",
    "start_time": "05:00",
    "duration": 5400.0
}

In [ ]:
{
    "program": {
        "uid": 6174211,
        "original_title": "Minutos musicales",
        "local_title": "Minutos musicales",
        "year": "2019",
        "channel": {
            "uid": 729036190,
            "name": "La Sexta",
            "country": "Spain"
        }
    }
}